<a href="https://colab.research.google.com/github/supsi-dacd-isaac/TeachDecisionMakingUncertainty/blob/main/L3_Robust_Optimization_example_production_proble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AN EXAMPLE OF ROBUSTIFIED LINEAR PROGRAM
Modified by:
 1. Erick Delage (Created for ROME 14 April 2015)
 2. Erick Delage (Adapted to RSOME in November 2020)
 3. Roberto Rocchetta (January 2025)

As discussed in example 2.5 of the  [lecture notes](http://tintin.hec.ca/pages/erick.delage/MATH80624_LectureNotes.pdf) of MATH80624 at HEC Montréal.


In [2]:
!pip install rsome
!pip install mosek
!rm mosek.lic
!git clone https://github.com/roberock/mosek_lic
!cp ./mosek_lic/mosek.lic .
!rm -r ./mosek_lic
!mkdir -p /root/mosek
!cp ./mosek.lic /root/mosek
#!pip install -i https://pypi.gurobi.com gurobipy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.8/87.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 53.0 MB/s eta 0:00:00
rm: cannot remove 'mosek.lic': No such file or directory
Cloning into 'mosek_lic'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


WARNING!!!

The following code uses a Personal Academic License file valid until **2026-jan-23**.

If you have error messages informing you about licencing issues, you may try uncommenting the installation lines for Gurobi.

Otherwise, we recommend that you obtain your own free accademic licence of either Mosek ([url](https://www.mosek.com/)) or Gurobi ([url](https://www.gurobi.com/)).

In [3]:
import rsome as rso
import numpy as np
from rsome import ro
from rsome import msk_solver as my_solver  #Import Mosek solver interface
#from rsome import grb_solver as my_solver  #Import Gurobi solver interface

For further details, see also the example 1.1.1 in the reference:

A. Ben-Tal, L. El Ghaoui, and A. Nemirovski. *Robust Optimization*. Princeton Series in Applied Mathematics, 2009.

## **Define a deterministic optimization problem**

Define a simple production problem for a company. A **deterministic optimal production model** is defined as follows:

\begin{align}
\max\limits_{\theta}\;& 6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2)\\
\text{subject to}\;&  m_1 + m_2 \leq 1000  && \text{(Storage)} \\
                 & 90n_1 + 100n_2 \leq 2000  && \text{(Manpower)} \\
                 & 40n_1 + 50n_2 \leq 800  && \text{(Equipment)} \\
                 & 100m_1+199.9m_2+700n_1+800n_2 \leq 100000 &&\text{(Budget)} \\
                 & 0.01m_1+0.02m_2-0.5n_1-0.6n_2 \geq 0 &&\text{(Catalyst/Agent A)} \\
                 &  m_1\geq0, m_2\geq0, n_1 \geq 0, n_2\geq 0.
\end{align}


**Descision variables:**
- $(n_1,n_2)$: number of boxes (1000 packs/box) produced for drug type 1 and type 2
- $(m_1, m_2)$: masses of raw material type 1 and Type 2

**Model coefficients:**

- $(c_1,c_2) = (6200, 6900)~~ [\frac{CHF}{Box}]$ expected revenue per box and drug type
- $(c_3,c_4) = (100, 199.9)~~ [\frac{CHF}{kg}]$: costs of raw materials type 1 and 2
- $(c_5,c_6) = (700, 800)~~ [\frac{CHF}{Box}]$: production cost per box and drug type



## **Solve a deterministic optimization problem**

In a compact form:

\begin{align}
\max\limits_{\theta\in\mathbb{R}^{+,n_\theta}}\;& c^T \theta \\
\text{subject to}\;&  A\theta  \leq b  &&  \\
\end{align}

**Objective function:**

- $6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2)=\\ \text{Reward}(\theta, c) - \text{Cost}(\theta, c)  = c^T \theta $

**Objective coefficients:**
- $c = [-(c_1-c_5),-(c_2-c_6), c_3, c_4]$

**decision variables:**
- $\theta = [n_1, n_2, m_1, m_2]$
   

In [4]:
from scipy.optimize import linprog

# Objective function coefficients (to be maximized)
c = [-(6200-700), -(6900-800), +100, +199.90]  # max J() --> min -J() Negated for maximization

# Inequality constraint matrix
A = [
    [0, 0, 1, 1],
    [90, 100, 0, 0],  # ManPower
    [40, 50, 0, 0],  # Equipment
    [700, 800, 100, 199.9],   # Budget
    [0.5, 0.6, -0.01, -0.02],  # Catalyst
]

# Inequality constraint vector
b = [1000,
     2000,
     800,
     100000,
     0]

def production_problem_det(c,A,b):
  # decision variables are non-negative
  bounds = [(0, None), (0, None), (0, None), (0, None)]

  # Solve the linear program
  return linprog(c, A_ub=A, b_ub=b, bounds=bounds)

#Run
result = production_problem_det(c,A,b)

# Determinisitc revenue
optobj_det = -result.fun

# Print the results
# print(result)

if result.success:
  print(" ---------------- ")
  print("Optimal Solution Deterministic Problem")
  print('Deterministic objective value')

  print(f"n1 =  {result.x[0]:.2f} boxes drug 1")
  print(f"n1 =  {result.x[1]:.2f} boxes drug 2")
  print(f"m1 =  {result.x[2]:.2f} [kg] Raw material 1")
  print(f"m2 =  {result.x[3]:.2f} [kg] Raw material 2")
  print(f"Objective Function Value = {optobj_det:.2f}  CHF") #negate to get the maximized value
else:
    print("Optimization failed: ", result.message)

 ---------------- 
Optimal Solution Deterministic Problem
Deterministic objective value
n1 =  17.55 boxes drug 1
n1 =  0.00 boxes drug 2
m1 =  0.00 [kg] Raw material 1
m2 =  438.79 [kg] Raw material 2
Objective Function Value = 8819.66  CHF


## **What could go wrong? Evaluate a production damage.**

We designed the production. We bought material $(m_1,m_2)$, a customer is going to buy $(n_1,n_2)$



**Unfortunately we realize that:**

- Error 1: there is a 2 % error on the estimation of the conversion rate 0.02 of this raw material 2.
- Error 2: there is a 0.5 % on the raw material 1.  

Hence, in the worst-case we will only get:
* 0.0196 kg of agent A per kg of raw material 2
* 0.0095 kg of agent A per kg of raw material 1.

**Try to:**

* Define the uncertainty set
  *  $\mathcal{U} = \{u\in\mathbb{R}^2: ....\}$
* Determine how much missing agent A in the worst case?
  * $w_{\Delta_A} = \min\limits_{u\in \mathcal{U}}[\text{Produced}(u,m_1,m_2)]-\text{Consumed}(n_1,n_2)$

* Define the problem with uncertain coefficients
  * $A(u)= ...$
* Evaluate the worst-case profit
  * $...$

In [8]:
def worst_case_production_error(m1, m2, n1, n2, error_yeld1 = 0.005, error_yeld2=0.02 ):
  """get the worst case missing Agent A (due to production errors u1, u2)"""

  # how much Agent A from the raw materials 1 and 2 ?
  produced_with_raw1 = (1-error_yeld1)*0.01*m1
  produced_with_raw2 = (1-error_yeld2)*0.02*m2

  # how much Agent A do we need to produce durgs 1 and 2 ?
  consumed_with_durg_1 = 0.5*n1
  consumed_with_durg_2 = 0.6*n2

  return -min(0, produced_with_raw1+produced_with_raw2-consumed_with_durg_1-consumed_with_durg_2)   # worst case yield [g] for agent A


det_n1, det_n2 = result.x[0],  result.x[1]
det_m1, det_m2  = result.x[2], result.x[3]
worst_case_deterministic_solution = worst_case_production_error(det_m1, det_m2,
                                                                det_n1, det_n2,
                                                                error_yeld1 = 0.005,
                                                                error_yeld2 = 0.02)


print(f'Deterministic solution: there could be up to {worst_case_deterministic_solution:.3f} [g] of missing agent A');


Deterministic solution: there could be up to 0.176 [g] of missing agent A


## **Evaluating the worst-case profit**


In [24]:
# Objective function coefficients (to be maximized)
c = [-(6200-700), -(6900-800), +100, +199.90]  # max J() --> min -J() Negated for maximization

# Inequality constraint matrix
A = [
    [0, 0, 1, 1],
    [90, 100, 0, 0],  # ManPower
    [40, 50, 0, 0],  # Equipment
    [700, 800, 100, 199.9],   # Budget
    [0.5, 0.6, -0.01*(1-0.005), -0.02*(1-0.02)],  # Catalyst
]

# Inequality constraint vector
b = [1000,
     2000,
     800,
     100000,
     0]

def production_problem_det(c,A,b):
  # decision variables are non-negative
  bounds = [(0, None), (0, None), (det_m1, det_m1), (det_m2, det_m2)] # we already ordered the raw material....fix it to known solution det_m1, det_m2
  # Solve the linear program
  return linprog(c, A_ub=A, b_ub=b, bounds=bounds)

#Run
result_wc = production_problem_det(c,A,b)

# Determinisitc revenue
optobj_det_wc = -result_wc.fun

# Print the results
# print(result_wc)

if result_wc.success:
  print(" ---------------- ")
  print("Solution Deterministic Problem with Uncertain Yield")
  print(f"n1 =  {result_wc.x[0]:.2f} boxes drug 1")
  print(f"n1 =  {result_wc.x[1]:.2f} boxes drug 2")
  print(f"m1 =  {result_wc.x[2]:.2f} [kg] Raw material 1")
  print(f"m2 =  {result_wc.x[3]:.2f} [kg] Raw material 2")
  print(f"Objective Function Value = {optobj_det_wc:.2f}  CHF")
else:
    print("Optimization failed: ", result_wc.message)

 ---------------- 
Solution Deterministic Problem with Uncertain Yield
n1 =  17.20 boxes drug 1
n1 =  0.00 boxes drug 2
m1 =  0.00 [kg] Raw material 1
m2 =  438.79 [kg] Raw material 2
Objective Function Value = 6888.99  CHF


In [27]:
# define the paramers (this is how much material we bought....mass RawI and mass RawII)
RawI = det_m1;
RawII = det_m2;

#Create model
model = ro.Model('production_raw_fixed_worst-case_scenario')

# Define decision variables
DrugI =model.dvar()             # Define a decision variable DrugI
DrugII =model.dvar()            # Define a decision variable DrugII

#Objective to maximize the profit
model.max(6200*DrugI+6900*DrugII - (100*RawI+199.90*RawII+700*DrugI+800*DrugII))

# Manpower constraint
model.st(90*DrugI + 100*DrugII <= 2000)

# Equipment constraint
model.st(40*DrugI+50*DrugII <= 800)

# Budget constraint
model.st(100*RawI+199.9*RawII+700*DrugI+800*DrugII <= 100000)

# Constraint to have enough active agent A under worst-case scenario
model.st(0.01*(1-0.005)*RawI+0.02*(1-0.02)*RawII-0.5*DrugI-0.6*DrugII >= 0)

# Constraints that decision variables are non-negative
model.st(DrugI >= 0)                             # DrugI is non-negative
model.st(DrugII >= 0)                            # DrugII is non-negative

# solve the model
model.solve(my_solver)

wc_det = model.get()
det_sol_wcDrugI   = DrugI.get()
det_sol_wcDrugII   = DrugII.get()

print('Same resut is obtained using the RSOME package with the Mosek solver')
print('Deterministic solution: Worst-case profit is',np.round(wc_det,2), '[CHF] vs ', np.round(optobj_det,2), '[CHF] of deterministic solution')
print('Deterministic solution: The unrealized profit is',np.round(wc_det-optobj_det,2), '[CHF] 😭')

print('Deterministic solution: Due to loss in production of', int(1000*np.round(det_n1- det_sol_wcDrugI,3)), ' packs of Drug I')
print('Deterministic solution: Due to loss in production of', int(1000*np.round(det_n2 - det_sol_wcDrugII,3)), ' packs of Drug II')

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0013s
Same resut is obtained using the RSOME package with the Mosek solver
Deterministic solution: Worst-case profit is 6888.99 [CHF] vs  8819.66 [CHF] of deterministic solution
Deterministic solution: The unrealized profit is -1930.67 [CHF] 😭
Deterministic solution: Due to loss in production of 351  packs of Drug I
Deterministic solution: Due to loss in production of 0  packs of Drug II


In [ ]:
# Can we do better?


# try your best, remeber that we now know what the yeld error is.....




## **Solving a robust model with a box uncertainty set**

Consider the following robust optimization model:
\begin{align}
\max\limits_{\theta}\;& 6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2) \\
\text{subject to}\;&  m_1 + m_2 \leq 1000  && \text{(Storage)} \\
                 & 90n_1 + 100n_2 \leq 2000  && \text{(Manpower)} \\
                 & 40n_1 + 50n_2 \leq 800  && \text{(Equipment)} \\
                 & 100m_1+199.9m_2+700n_1+800n_2 \leq 100000 &&\text{(Budget)} \\
                 & 0.01(1-0.005u_1)m_1+0.02(1-0.02u_2)m_2-0.5n_1-0.6n_2 \geq 0   \;\forall\,(u_1,u_2)\in\mathcal{U} && \text{(Robsut constraint produced Agent A)} \\
                 &  m_1\geq0, m_2\geq0, n_1 \geq 0, n_2\geq 0.
\end{align}

where $\mathcal{U}$ is a box-shaped uncertainty set: $$\mathcal{U}:=\{u\in\mathbb{R}^2 : |u_i| \leq 1, \forall i=1, 2\}.$$




**In a compact form:**

\begin{align}
\max\limits_{\theta\in\mathbb{R}^{+,n_\theta}}\;& c^T \theta \\
\text{subject to}\;&  A(u)\theta  \leq b,  &&   \;\forall\, u\in\mathcal{U} \\
\end{align}




In [11]:
#Create model
model = ro.Model('robustproduction_box')

max_error_1 = 0.005
max_error_2 = 0.02

# Define decision variables
n1 =model.dvar()              # Define a decision variable for n boxes of Drug I
n2 =model.dvar()              # Define a decision variable for n boxes of Drug II
m1 =model.dvar()              # Define a decision variable mass Raw I
m2 =model.dvar()              # Define a decision variable mass Raw II

# Declare uncertain parameters and the box uncertainty set
u = model.rvar(2)           # define 2 random variables
boxSet = (abs(u) <= 1);     # each laing within a box [-1, 1]

# Objective to maximize the profit
model.max(6200*n1+6900*n2 - (100*m1+199.90*m2+700*n1+800*n2))

# Constraints
model.st(m1 + m2 <= 1000)                         # Storage constraint
model.st(90*n1 + 100*n2 <= 2000)                  # Manpower constraint
model.st(40*n1+50*n2 <= 800)                      # Equipment constraint
model.st(100*m1+199.9*m2+700*n1+800*n2 <= 100000) # Budget constraint

# Constraints that decision variables are non-negative
model.st(n1 >= 0)           # DrugI is non-negative
model.st(n2 >= 0)           # DrugII is non-negative
model.st(m1 >= 0)           # RawI is non-negative
model.st(m2 >= 0)           # RawII is non-negative

# Constraint to have enough active agent A
model.st((0.01*(1-max_error_1*u[0])*m1+0.02*(1-max_error_2*u[1])*m2-0.5*n1-0.6*n2 >= 0).forall(boxSet))

# solve the model
model.solve(my_solver)

optobj_rob = model.get()      #get optimal objective value
rob_sol_DrugI   = n1.get()    #get optimal solution
rob_sol_DrugII  = n2.get()    #get optimal solution
rob_sol_RawI    = m1.get()    #get optimal solution
rob_sol_RawII   = m2.get()    #get optimal solution

print('Robust solution: Expected profit is', np.round(optobj_rob,2), 'CHF')
print('Robust solution: Robust Solution Raw Material I = ', np.round(rob_sol_RawI,2))
print('Robust solution: Robust Solution Raw Material II = ', np.round(rob_sol_RawII,2))
print('Robust solution: Robust Solution Drug I = ', np.round(rob_sol_DrugI,2))
print('Robust solution:  Robust Solution Drug II = ', np.round(rob_sol_DrugII,2))

print('When drifts are as small in a BOX set it is possible to avoid a potential loss of', "{0:.0f}".format((optobj_det-wc_det)/optobj_det*100),'% by giving away', "{0:.1f}".format((optobj_det-optobj_rob)/optobj_det*100),'% of profit.⚖️' )

Being solved by Mosek...
Solution status: Optimal
Running time: 0.0023s
Robust solution: Expected profit is 8294.57 CHF
Robust solution: Robust Solution Raw Material I =  877.73
Robust solution: Robust Solution Raw Material II =  0.0
Robust solution: Robust Solution Drug I =  17.47
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a BOX set it is possible to avoid a potential loss of 22 % by giving away 6.0 % of profit.⚖️


In [18]:
# show that we are never missing [g] of Agent A
for k in range(5):
  u = np.random.uniform(-1,1,2)  # explore different yield errors in [-1,+1]
  error_yeld1 = max_error_1*u[0]
  error_yeld2 = max_error_2*u[1]
  worst_case_robust = worst_case_production_error(m1=rob_sol_RawI,
                                                m2=rob_sol_RawII,
                                                n1=rob_sol_DrugI,
                                                n2=rob_sol_DrugII,
                                                error_yeld1 = error_yeld1,
                                                error_yeld2 = error_yeld2)

  print(f'Missing agent A for u={u} is {worst_case_robust:.3f} [g] of missing agent A');





Missing agent A for u=[ 0.7052201  -0.48584805] is 0.000 [g] of missing agent A
Missing agent A for u=[ 0.52809134 -0.42036403] is 0.000 [g] of missing agent A
Missing agent A for u=[0.01554329 0.79696614] is 0.000 [g] of missing agent A
Missing agent A for u=[0.69958603 0.71891386] is 0.000 [g] of missing agent A
Missing agent A for u=[-0.11477528  0.23034576] is 0.000 [g] of missing agent A


In [19]:
# WE ARE NOT GUARANTEED OUTSIDE FROM THE UNCERTAINTY SET (missing agent A)
for k in range(5):

  u = np.random.uniform(-2,2,2) # clearly, a wrong characterization of the error can lead to some issues
  error_yeld1 = max_error_1*u[0]
  error_yeld2 = max_error_2*u[1]
  worst_case_robust = worst_case_production_error(m1=rob_sol_RawI,
                                                m2=rob_sol_RawII,
                                                n1=rob_sol_DrugI,
                                                n2=rob_sol_DrugII,
                                                error_yeld1 = error_yeld1,
                                                error_yeld2 = error_yeld2)

  print(f'Missing agent A for u={u} is {worst_case_robust:.4f} [g] of missing agent A');

Missing agent A for u=[-1.98459261  0.59559218] is 0.0000 [g] of missing agent A
Missing agent A for u=[1.26657055 0.59364459] is 0.0117 [g] of missing agent A
Missing agent A for u=[-1.39365723  1.74940238] is 0.0000 [g] of missing agent A
Missing agent A for u=[-0.0922356  -1.01402272] is 0.0000 [g] of missing agent A
Missing agent A for u=[1.82155234 1.74347073] is 0.0361 [g] of missing agent A


## **Solving a robust model with an ellipsoidal uncertainty set**

Consider the following robust optimization model with an ellipsoidal uncertainty set:

\begin{align}
\max\limits_{\theta\in\mathbb{R}^{+,n_\theta}}\;& c^T \theta \\
\text{subject to}\;&  A(u)\theta  \leq b,  &&   \;\forall\, u\in\mathcal{U} \\
\end{align}

where $\mathcal{U}:=\left\{u\in\mathbb{R}^2\;\middle|\;  ||u||_2 = \sqrt{u_1^2+u_2^2} \leq 1\right\}.$

In [28]:
#Create model
model = ro.Model('robustproduction_elipsoidal')

max_error_1 = 0.005
max_error_2 = 0.02

# Define decision variables
n1 =model.dvar()              # Define a decision variable for n boxes of Drug I
n2 =model.dvar()              # Define a decision variable for n boxes of Drug II
m1 =model.dvar()              # Define a decision variable mass Raw I
m2 =model.dvar()              # Define a decision variable mass Raw II

# Declare uncertain parameters and the box uncertainty set
u = model.rvar(2)            # define 2 random variables
BallSet= (rso.norm(u,2)<=1)  # the perturbations are within a ball of radius 1


# Objective to maximize the profit
model.max(6200*n1+6900*n2 - (100*m1+199.90*m2+700*n1+800*n2))

# Constraints
model.st(m1 + m2 <= 1000)                         # Storage constraint
model.st(90*n1 + 100*n2 <= 2000)                  # Manpower constraint
model.st(40*n1+50*n2 <= 800)                      # Equipment constraint
model.st(100*m1+199.9*m2+700*n1+800*n2 <= 100000) # Budget constraint

# Constraints that decision variables are non-negative
model.st(n1 >= 0)           # DrugI is non-negative
model.st(n2 >= 0)           # DrugII is non-negative
model.st(m1 >= 0)           # RawI is non-negative
model.st(m2 >= 0)           # RawII is non-negative

# Constraint to have enough active agent A
model.st((0.01*(1+max_error_1*u[0])*m1+0.02*(1+max_error_2*u[1])*m2-0.5*n1-0.6*n2 >= 0).forall(BallSet))

# solve the model with my_solver
model.solve(my_solver)

optobj_rob = model.get()      #get optimal objective value
rob_sol_DrugI   = n1.get()    #get optimal solution
rob_sol_DrugII  = n2.get()    #get optimal solution
rob_sol_RawI    = m1.get()    #get optimal solution
rob_sol_RawII   = m2.get()    #get optimal solution

print('Robust solution: Expected profit is', np.round(optobj_rob,2), 'CHF')
print('Robust solution: Robust Solution Raw Material I = ', np.round(rob_sol_RawI,2))
print('Robust solution: Robust Solution Raw Material II = ', np.round(rob_sol_RawII,2))
print('Robust solution: Robust Solution Drug I = ', np.round(rob_sol_DrugI,2))
print('Robust solution:  Robust Solution Drug II = ', np.round(rob_sol_DrugII,2))
print('When drifts are as small in a BALL set it is possible to avoid a potential loss of', "{0:.0f}".format((optobj_det-wc_det)/optobj_det*100),'% by giving away', "{0:.1f}".format((optobj_det-optobj_rob)/optobj_det*100),'% of profit.⚖️' )


Being solved by Mosek...
Solution status: Optimal
Running time: 0.0026s
Robust solution: Expected profit is 8311.76 CHF
Robust solution: Robust Solution Raw Material I =  821.12
Robust solution: Robust Solution Raw Material II =  28.31
Robust solution: Robust Solution Drug I =  17.47
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a BALL set it is possible to avoid a potential loss of 22 % by giving away 5.8 % of profit.⚖️


## **Solving a robust model with a budgeted uncertainty set**


Consider the following robust optimization model:
\begin{align}
\max\limits_{\theta}\;& 6200n_1+6900n_2-(100m_1+199.90m_2+700n_1+800n_2) \\
\text{subject to}\;&  m_1 + m_2 \leq 1000  && \text{(Storage)} \\
                 & 90n_1 + 100n_2 \leq 2000  && \text{(Manpower)} \\
                 & 40n_1 + 50n_2 \leq 800  && \text{(Equipment)} \\
                 & 100m_1+199.9m_2+700n_1+800n_2 \leq 100000 &&\text{(Budget)} \\
                 & 0.01(1-0.005u_1)m_1+0.02(1-0.02u_2)m_2-0.5n_1-0.6n_2 \geq 0   \;\forall\,(u_1,u_2)\in\mathcal{U} && \text{(Robsut constraint produced Agent A)} \\
                 &  m_1\geq0, m_2\geq0, n_1 \geq 0, n_2\geq 0.
\end{align}

where $\mathcal{U}$ is a box-shaped uncertainty set: $$\mathcal{U}:=\{(u_1,u_2) \in\mathbb{R}^2\;:\; \sum\limits_{i=1}^2|z_i| \leq \Gamma, |z_i| \leq 1, \forall i =1,2\}.$$

Or in a compact form:
 $$\mathcal{U}:=\{u\in\mathbb{R}^2\;:\; ||z||_{1} \leq \Gamma, ||z||_{\infty}  \leq 1\}.$$


In [29]:
#Create model
model = ro.Model('robustproduction_budgeted')

# Define decision variables
n1 =model.dvar()              # Define a decision variable for n boxes of Drug I
n2 =model.dvar()              # Define a decision variable for n boxes of Drug II
m1 =model.dvar()              # Define a decision variable mass Raw I
m2 =model.dvar()              # Define a decision variable mass Raw II


# Declare uncertain parameters
Gamma = 1
u = model.rvar(2)
budgetedSet=(abs(u) <= 1, rso.norm(u, 1) <=Gamma)  # define the budgeted uncertainty set

# Objective to maximize the profit
model.max(6200*n1+6900*n2 - (100*m1+199.90*m2+700*n1+800*n2))

# Constraints
model.st(m1 + m2 <= 1000)                         # Storage constraint
model.st(90*n1 + 100*n2 <= 2000)                  # Manpower constraint
model.st(40*n1+50*n2 <= 800)                      # Equipment constraint
model.st(100*m1+199.9*m2+700*n1+800*n2 <= 100000) # Budget constraint

# Constraints that decision variables are non-negative
model.st(n1 >= 0)           # DrugI is non-negative
model.st(n2 >= 0)           # DrugII is non-negative
model.st(m1 >= 0)           # RawI is non-negative
model.st(m2 >= 0)           # RawII is non-negative


# Constraint to have enough active agent A
model.st((0.01*(1+0.005*u[0])*m1+0.02*(1+0.02*u[1])*m2-0.5*n1-0.6*n2 >= 0).forall(budgetedSet))

# solve the model
model.solve(my_solver)

optobj_rob = model.get()      #get optimal objective value
rob_sol_DrugI   = n1.get()    #get optimal solution
rob_sol_DrugII  = n2.get()    #get optimal solution
rob_sol_RawI    = m1.get()    #get optimal solution
rob_sol_RawII   = m2.get()    #get optimal solution

print('Robust solution: Expected profit is', np.round(optobj_rob,2), 'CHF')
print('Robust solution: Robust Solution Raw Material I = ', np.round(rob_sol_RawI,2))
print('Robust solution: Robust Solution Raw Material II = ', np.round(rob_sol_RawII,2))
print('Robust solution: Robust Solution Drug I = ', np.round(rob_sol_DrugI,2))
print('Robust solution:  Robust Solution Drug II = ', np.round(rob_sol_DrugII,2))
print('When drifts are as small in a budgeted set it is possible to avoid a potential loss of', "{0:.0f}".format((optobj_det-wc_det)/optobj_det*100),'% by giving away', "{0:.1f}".format((optobj_det-optobj_rob)/optobj_det*100),'% of profit.⚖️' )


Being solved by Mosek...
Solution status: Optimal
Running time: 0.0017s
Robust solution: Expected profit is 8399.6 CHF
Robust solution: Robust Solution Raw Material I =  702.16
Robust solution: Robust Solution Raw Material II =  87.77
Robust solution: Robust Solution Drug I =  17.48
Robust solution:  Robust Solution Drug II =  0.0
When drifts are as small in a budgeted set it is possible to avoid a potential loss of 22 % by giving away 4.8 % of profit.⚖️
